<a href="https://colab.research.google.com/github/ktjylsj/save/blob/master/L03_Yolov3(tiny)_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**1. Connect Google Drive to access training dataset**

본인 계정의 Google Drive 를 mount 합니다. 이를 통해 file 들을 Google Drive 에 save / load 합니다.<p>
Training 에 장시간이 소요되기에 트레이닝 데이터 백업 등에 Google Drive 를 사용하는 것이 좋다. 세부 수행 순서는 아래와 같다.
1. 실행 (Google Log-in, Google 계정이 없다면 생성해야 함)
2. Go to this URL in a browser 옆의 링크 클릭
3. 새로 생성된 web browser 에서 본인 google 계정 선택
4. Access 허용
5. 생성된 코드 복사
6. 아래 Enter your authorization code 에 붙여넣고 enter 버튼을 누른다.
7. Mounted at /content/drive 라고 출력되면 된다. 

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
!ls /content/drive/MyDrive

## **2. Upload your dataset**

### **2.1. Google Driver 를 /data 폴더로 symbolic link 로 연결한다.** 

In [ ]:
!rm -rf /data
!mkdir -p data
!mkdir -p backup

!ln -s "/content/drive/MyDrive" /data


### **2.2. Training 에 필요한 dataset 을 upload 한다.**

이전 과정에서 작업한 파일을 업로드 하면 되며 Downloads 폴더 아래 위치에 있는 파일을 upload 하자.
> task_test-xxx-yolo 1.1.zip


In [ ]:
from google.colab import files
uploaded = files.upload()

Upload 가 완료되면 압축을 풀어준다.

In [ ]:
!ls -la
!unzip 'task*.zip'

In [ ]:
!pwd && ls
!cd obj_train_data && ls

## **3. Generate required files for YOLOv3**

In [ ]:
cd /content/obj_train_data

아래 코드는 Ulload 된 dataset 파일들을 Yolo training 에 맞는 형식으로 변환해 준다.<p>
임의로 upload 된 dataset 의 70% 는 training 에, 나머지 30% 는 validaiton 에 사용한다. 


In [ ]:
import glob
import os

current_dir = os.path.abspath('')
print("Current DIR:", current_dir)

img_dir = os.path.join(current_dir)

file_train = open('train.txt', 'w')
file_valid = open('valid.txt', 'w')
file_datas = open('obj.data', 'w')

counter = 1
percentage_test = 30
index_test = round(100 / percentage_test)

for pathAndFilename in glob.iglob(os.path.join(img_dir, "*.jpg")):
    title, ext = os.path.splitext(os.path.basename(pathAndFilename))

    if counter == index_test:
        file_valid.write(img_dir + "/" + title + '.jpg' + "\n")
        counter = 1
    else:
        file_train.write(img_dir + "/" + title + '.jpg' + "\n")
        counter = counter + 1

file_datas.write("""\
classes = 1
train = {0}/train.txt
valid = {0}/valid.txt
names = {0}/obj.names
backup = /data/
""".format(current_dir))

file_train.close()
file_valid.close()
file_datas.close()

In [ ]:
!cp /content/obj.names /content/obj_train_data/
!cat /content/obj_train_data/obj.names

In [ ]:
!cat /content/obj_train_data/obj.data

In [ ]:
!cat /content/obj_train_data/valid.txt

## **4. Clone and Build Darknet**

Yolo training 에 사용될 코드를 git clone 으로 내려받고 빌드한다. 이때 build option 으로 GPU 를 사용하게 설정한다. 

In [ ]:
cd /content

In [ ]:
!rm -rf darknet && git clone https://github.com/AlexeyAB/darknet.git
!sed -i -e 's/GPU=0/GPU=1/g' -e 's/CUDNN=0/CUDNN=1/g' -e 's/OPENCV=0/OPENCV=1/g' darknet/Makefile
!cd darknet && make 2>/dev/null

Build 되는 동안 yolo training 에 사용될 darknet 기반의 코드를 살펴보자. https://github.com/AlexeyAB/darknet.git

## **5. Start Training**

In [ ]:
cd /data

In [ ]:
!nvidia-smi

**"yolov3-tiny-clp.cfg"** 파일과 **"yolov3-tiny-clp_con.weights"**: 파일 각각을 google drive 에 upload 한다.<p>
> 파일 위치: **\<your demo path\>/resources/tollgate**

아래 colab 에서 upload 하는 방법보다는 google drive 에 바로 upload 하는 것을 속도 측면에서 권장함 >> https://drive.google.com/

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!ls

In [ ]:
!cat yolov3-tiny-clp.cfg

In [ ]:
!ls -lah /content/drive/MyDrive

실제 darknet 기반으로 Yolo v3 tiny 의 모델 training 을 시작한다. pre-train 된 모델을 기반으로 추가적인 training 을 하기 때문에 약 20분 내외의 시간이 소요된다.

In [ ]:
#!cd /content/darknet && ./darknet detector train ../obj_train_data/obj.data /data/yolov3-tiny-clp.cfg /data/yolov3-tiny.conv.15 -dont_show -clear
!cd /content/darknet && ./darknet detector train ../obj_train_data/obj.data /data/yolov3-tiny-clp.cfg /data/yolov3-tiny-clp_con.weights -dont_show -clear

## **6. Convert YOLO* Models to TensorFlow Model**

OpenVINO 2021.2 버전을 설치한다.

In [ ]:
cd /content

In [ ]:
!wget https://apt.repos.intel.com/openvino/2021/GPG-PUB-KEY-INTEL-OPENVINO-2021
!apt-key add GPG-PUB-KEY-INTEL-OPENVINO-2021
!touch /etc/apt/sources.list.d/intel-openvino-2021.list
!echo "deb https://apt.repos.intel.com/openvino/2021 all main" >> /etc/apt/sources.list.d/intel-openvino-2021.list
!apt update
#!apt-cache search intel-openvino-dev-ubuntu18
!apt install intel-openvino-dev-ubuntu18-2021.2.200

TensorFlow v1.x 를 설치한다. 

In [ ]:
!cd /opt/intel/openvino_2021/deployment_tools/model_optimizer/install_prerequisites && ./install_prerequisites_tf.sh

Darknet model 을 TF 로 변환하기 위한 github repository 를 다운로드 한다. 

In [ ]:
!git clone https://github.com/mystic123/tensorflow-yolo-v3.git
!cd tensorflow-yolo-v3 && git checkout ed60b90

Training 한 darknet 기반의 최종 결과물을 TF 의 frozen 된 모델로 변환 후 google drive 에 저장한다.<br> 
차후 IR 로 변환하기 위해 google drive 에서 "**frozen_darknet_yolov3_model.pb**" 를  로컬로 다운받아 사용하면 된다. 

In [ ]:
!cd tensorflow-yolo-v3 && \
python3 convert_weights_pb.py --class_names /content/obj_train_data/obj.names --data_format NHWC --weights_file /data/yolov3-tiny-clp_final.weights --tiny

In [ ]:
!ls /data/yolo*


In [ ]:
!cp /content/tensorflow-yolo-v3/frozen_darknet_yolov3_model.pb /data

 Tensorflow frozen format 으로 변환이 완료된 네트워크 모델 (frozen_darknet_yolov3_model.pb) 을 google drive 에서 로컬 PC 로 다운로드 한다. <p>
 아래 colab 에서 download 하는 방법보다는 google drive 에 바로 download 하는 것을 속도 측면에서 권장함 >> https://drive.google.com/